In [1]:
import pandas as pd

In [4]:
participants_file = '/Users/yasaman/Dropbox/UW/Make4All/UWEXP Phase II/EMA/compliance/participants.csv'
database_file = '/Users/yasaman/Dropbox/UW/Make4All/UWEXP Phase II/EMA/compliance/database.csv'
qualtrics_file = '/Users/yasaman/Dropbox/UW/Make4All/UWEXP Phase II/EMA/compliance/qualtrics.csv'
complinace_file = '/Users/yasaman/Dropbox/UW/Make4All/UWEXP Phase II/EMA/compliance/compliance.csv'
survey_date = '2019-04-14'
survey_start = '10am'
survey_end = '10pm'

In [5]:
participants = pd.read_csv(participants_file)
database = pd.read_csv(database_file)
qualtrics = pd.read_csv(qualtrics_file, header=0, skiprows=[1])

In [6]:
# identify active PID's with no device ID
participants.loc[(~participants['PID'].isin(database['PID'])) & (participants['Status'] == 'ACTIVE'), 'Status'] = 'NO DEVICE_ID'

In [7]:
# convert start and end information to datetime objects
qualtrics[['startDate', 'endDate']] = qualtrics[['startDate', 'endDate']].apply(pd.to_datetime)

In [8]:
# obtain the responses valid for this survey
from_ = pd.to_datetime(survey_date + ' ' +survey_start)
to_ = pd.to_datetime(survey_date + ' ' +survey_end)

valid = (qualtrics['startDate'] >= from_) & (qualtrics['endDate'] <= to_) 
valid_responses = qualtrics[valid][['startDate', 'endDate', 'device_id']]

In [9]:
num_resps = valid_responses.groupby(by=['device_id']).size()
num_resps[num_resps > 1]

device_id
3fc70ec4-ec42-4e3b-a772-313fa25d7531    2
56941fb4-9b18-46d0-86b5-42bb7a35439d    2
815db9c4-a652-4b03-8f16-5facdc81bcdd    2
a65c7f97-21e0-463f-9a76-bfc559b620c5    2
c7d3d0cf-e58e-403b-a8bd-1b461ae4ee5b    2
cbdb8590-f055-450d-82ab-f6c3ff7ad2e4    2
f9e4f7af-34a0-45dd-bf51-88aea9ff81c1    2
dtype: int64

In [ ]:
# TO-DO remove near empty respnses

In [10]:
valid_responses = valid_responses.groupby(by=['device_id']).first().reset_index()

In [11]:
# obtain the PID for valid responses
pids = pd.merge(valid_responses, database, on = 'device_id', how='left')[['device_id', 'PID']].sort_values(by=['PID'])

In [12]:
status = pd.merge(participants, pids, on='PID', how='left')


In [13]:
status.loc[(status['Status'] == 'ACTIVE') & (status['device_id'].isnull()), 'Status'] = 'MISSING'

In [14]:
status.loc[(status['Status'] == 'NO DEVICE_ID') | (status['Status'] == 'MISSING'), survey_date] = 'missing'
status.loc[(status['Status'] == 'DROPPED') , survey_date] = 'DROPPED'

In [15]:
status.to_csv(complinace_file, index=False)